In [31]:
#Import required packages
from torchdiffeq import odeint_adjoint as odeint
import numpy as np 
import torch.nn as nn
import torch.optim as optim

In [32]:
#Instantiate functions to compute activation and repression equations
def activation(x, k, theta, n):
    return (k*(x/theta)**n)/(1+(x/theta)**n)

def repression(x, k, theta, n):
    return k/(1+(x/theta)**n)

def nonlinearity(x, kc, km):
    return (kc*x)/(km+x)

In [33]:
#Define custom Module with fixed dual control architecture
class DualControl(nn.Module):
    def __init__(self):
        '''Constructor instantiating weights as model parameters and constants'''
        super().__init__()
        
        #Initialize constants, taken from Verma et al paper.
        self.Vin = 1.
        self.e0 = 0.0467
        self.lam = 1.93E-4 #1/s
        #Assume equal kinetics for all three enzymes
        self.kc = 12
        self.km = 10 #1/s

        #Initizalize weights for training
        self.W = nn.Parameter(torch.from_numpy(np.array([[2,2],[1,1], [1E-7, 1E-7]]))) 
        #parameters are n1, n2, theta1, theta2, k1, k2
        
    def forward(self, t, y):
        '''Computes derivatives of system of differential equations'''
        ydot = torch.zeros(6)
        ydot[0] = self.Vin - self.lam*y[0] - self.e0*nonlinearity(y[0], self.kc, self.km) - self.lam*y[1]
        ydot[1] = y[2]*nonlinearity(y[0], self.kc, self.km) - y[3]*nonlinearity(y[1], self.kc, self.km) - self.lam*y[1]
        ydot[2] = repression(y[1], self.W[2][0], self.W[1][0], self.W[0][0]) - self.lam*y[2]
        ydot[3] = activation(y[1], self.W[2][1], self.W[1][1], self.W[0][1]) - self.lam*y[3]
        ydot[4] = (self.Vin -  y[3]*nonlinearity(y[1], self.kc, self.km))**2
        ydot[5] = repression(y[1], self.W[2][0], self.W[1][0], self.W[0][0]) + activation(y[1], self.W[2][1], self.W[1][1], self.W[0][1])
        return ydot

In [34]:
#Custom loss function
def my_loss(solution, alpha=1):
    """Computes scalarized loss including genetic constraint and product production"""
    j1 = solution[-1][-2].clone().detach().requires_grad_(True)
    j2 = solution[-1][-1].clone().detach().requires_grad_(True)
    loss = j1 + alpha*j2
    return loss

In [35]:
#Establish initial conditions
y0 = torch.from_numpy(np.array([2290., 0., 0., 0., 0., 0.]))
t = torch.from_numpy(np.linspace(1,5e4,200))

#Construct model by instantiating class defined above
model = DualControl()

#Set hyperparameters
learning_rate = 0.01
weight_decay_coefficient = 0
num_epochs = 10
criterion = my_loss

#Establish optimizer - settings from torchdiffeq package examples
optimizer = optim.Adam(model.parameters(), amsgrad=False,
                                    weight_decay=weight_decay_coefficient, lr=learning_rate)
                
for i in range(num_epochs):

    # Forward pass: Compute steady-state solution
    y_pred = odeint(model, y0, t, method='dopri5')

    # Compute and print loss
    loss = criterion(y_pred)
    if i % 10 == 9 or True:
        print('Epoch',  i, 'Loss', loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch 0 Loss 49924.188941071945
Epoch 1 Loss 49924.188941071945
Epoch 2 Loss 49924.188941071945
Epoch 3 Loss 49924.188941071945
Epoch 4 Loss 49924.188941071945
Epoch 5 Loss 49924.188941071945
Epoch 6 Loss 49924.188941071945
Epoch 7 Loss 49924.188941071945
Epoch 8 Loss 49924.188941071945
Epoch 9 Loss 49924.188941071945
